In [451]:
import numpy as np
import math as m
from collections import defaultdict

In [454]:
class Point:
    __slots__ = 'x', 'y'
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "{" + str(self.x) + "," + str(self.y) + "}"
    
    def __repr__(self):
        return self.__str__()
    
    def dist(self, other):
        return abs(self.x-other.x)+abs(self.y-other.y)

class Ride:
    __slots__ = 'index', 'start', 'end', 'start_time', 'end_time', 'len', 'taken'
    
    def __init__(self, index, start, end, start_time, end_time):
        self.index = index
        self.start = Point(start[0], start[1])
        self.end = Point(end[0], end[1])
        self.start_time = start_time
        self.end_time = end_time
        self.len = self.start.dist(self.end)
        self.taken = False
        
    def can_finish_in_time(self, vehicle, time):
        distance = vehicle.position.dist(self.start) + self.len
        return time + distance <= self.end_time 
    
    def __str__(self):
        return "Ride number {}".format(self.index)
    
    def __repr__(self):
        return "{}| {}({})->{}({}) = {}".format(self.index, self.start, self.start_time, self.end, self.end_time, self.len)

    
class Vehicle:
    __slots__ = 'index', 'position', 'routes', 'time'
        
    def __init__(self, index):
        self.index = index
        self.position = Point(0, 0)
        self.time = 0
        self.routes = []

        
    def take_the_best_route(self, rides, max_time, bonus):
        if self.time >= max_time:
            return False
        
        max_score = None
        the_best_end_time = 0
        index = -1
        for i, ride in enumerate(rides):
            if ride.taken or not ride.can_finish_in_time(self, self.time):
                continue
            
            distance = ride.start.dist(self.position)
            start_time = max(ride.start_time, self.time+distance)
            end_time = start_time + ride.len
            waiting_time = start_time - self.time
            score = 0.005*ride.len - 2*waiting_time - distance 
#             if start_time == ride.start_time:
#                 score += bonus
            
            if max_score is None or score > max_score: # The best found
                max_score = score
                the_best_end_time = end_time
                index = i
                
        if index != -1:
            chosen = rides[index]
            self.routes.append(chosen.index)
            self.position = chosen.end
            self.time = the_best_end_time
            
            chosen.taken = True
            return True
        
        return False

In [455]:
names = [
    'a_example',
    'b_should_be_easy',
    'c_no_hurry',
    'd_metropolis',
    'e_high_bonus'
]
file_name = names[3]

In [456]:
def load_input_file(fileName):
    with open(fileName) as f:
        settings = [int(i) for i in f.readline().split()]
        input_data = []
        for line in f:
            input_data.append([int(x) for x in (line[:-1].split(' '))])
        ret = []
        for i, l in enumerate(input_data):
            ret.append(Ride(i, (l[0], l[1]), (l[2], l[3]), l[4], l[5]))

        ret_s = {
            'rows': settings[0],
            'cols': settings[1],
            'vehicles': settings[2],
            'rides': settings[3],
            'bonus': settings[4],
            'steps': settings[5]
        }
        return ret_s, ret
   
settings, rides = load_input_file(file_name + '.in')

In [457]:
vehicles = [Vehicle(i) for i in range(settings['vehicles'])]

In [458]:
time = settings['steps']
for vehicle in vehicles:
    while vehicle.take_the_best_route(rides, time, settings['bonus']):
        pass

In [459]:
with open(file_name + '.out', "w") as text_file:
    for vehicle in vehicles:
        rides = " ".join([str(r) for r in vehicle.routes])
        line = "{} {}\n".format(len(vehicle.routes), rides)
        text_file.write(line)